<a href="https://colab.research.google.com/github/AnupJoseph/learn-pytorch/blob/master/LearnPytorch3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()

    # Define kernel layers
    self.conv1 = nn.Conv2d(1,6,3)
    self.conv2 = nn.Conv2d(6,16,3)

    # Affine computations i.e Y = Wx + b
    self.fc1 = nn.Linear(16*6*6*1,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def num_flat_features(self,image):
    size = image.size()[1:]
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

  def forward(self,image):
    image = F.max_pool2d(F.relu(self.conv1(image)),(2,2))
    image = F.max_pool2d(F.relu(self.conv2(image)),2)
    image = image.view(-1,self.num_flat_features(image))
    image = F.relu(self.fc1(image))
    image = F.relu(self.fc2(image))
    image = self.fc3(image)
    return image

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [8]:
# To find the learnable parameters
params = list(net.parameters())
print(params[0].size())

torch.Size([6, 1, 3, 3])


In [9]:
# testing

inp = torch.randn(1,1,32,32)
out = net(inp)
print(out)

tensor([[-0.0549,  0.0931,  0.0364,  0.0488,  0.0433, -0.0232,  0.0390,  0.1188,
          0.0680,  0.0241]], grad_fn=<AddmmBackward>)


In [0]:
# Computing gradients
net.zero_grad()
out.backward(torch.randn(1,10))

In [13]:
# Computing losses
out = net(inp)
target = torch.randn(10)  # Dummy target
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(target,out)
print(loss)

tensor(0.7025, grad_fn=<MeanBackward0>)


In [14]:
# Implementing backprop

net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0125, -0.0057, -0.0030,  0.0056, -0.0019,  0.0157])


In [0]:
# Using torch.optim to update loss functions

import torch.optim as optm

optimizer = optm.SGD(net.parameters(),lr=0.02)

optimizer.zero_grad()
outkut = net(inp)

loss = criterion(outkut,target)

loss.backward()
optimizer.step()